In [ ]:
!pip install umap-learn
!pip install hdbscan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.4 MB/s eta 0:00:00
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Successfully uninstalled Cython-3.0.10


In [ ]:
import numpy as np
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.matutils import sparse2full
from sklearn.cluster import KMeans
#import umap.umap_ as umap
#import hdbscan

In [ ]:
texts  = ["Looking for a software engineer proficient in Python and JavaScript to work on full-stack development with experience in Django and React.", "Seeking a marketing specialist with expertise in SEO, content strategy, and social media management to develop and execute digital marketing campaigns.", "Hiring a financial analyst to perform data-driven analysis, generate financial reports, and assist in strategic decision-making.", "Looking for a machine learning engineer experienced in building models using TensorFlow and PyTorch to work on predictive analytics projects.", "We are looking for a project manager with PMP certification to oversee software development teams and ensure project delivery within deadlines.", "Seeking a UX/UI designer to create engaging, user-friendly interfaces, with strong skills in Figma and Adobe XD.", "Hiring a data scientist with experience in statistical modeling, data mining, and visualization tools like Tableau and Power BI.", "Looking for a content writer to produce high-quality blog posts, articles, and copy for websites, with excellent research and writing skills.", "Seeking an HR manager to handle employee relations, talent acquisition, and performance management for a fast-growing tech startup.", "We are hiring a customer support specialist to manage client queries and provide exceptional service through multiple channels, including chat, phone, and email."]

In [ ]:
tokenized_texts = [text.lower().split() for text in texts]
dictionary = Dictionary(tokenized_texts)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

In [ ]:
tfidf = TfidfModel(corpus)
tfidf_vectors = np.array([sparse2full(tfidf[dictionary.doc2bow(text)], len(dictionary)) for text in tokenized_texts])

In [ ]:
n_clusters = 5 # Desired number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
cluster_labels = kmeans.fit_predict(tfidf_vectors)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
def print_clusters(texts, labels, title):
    clusters = {}
    for i, label in enumerate(labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(texts[i])

    print(f"\n{title} Clusters:")
    for label, sentences in clusters.items():
        print(f"Cluster {label}:")
        for sentence in sentences:
            print(f"  {sentence}")

print_clusters(texts, cluster_labels, 'TFIDF')


TFIDF Clusters:
Cluster 3:
  Looking for a software engineer proficient in Python and JavaScript to work on full-stack development with experience in Django and React.
  Looking for a machine learning engineer experienced in building models using TensorFlow and PyTorch to work on predictive analytics projects.
Cluster 4:
  Seeking a marketing specialist with expertise in SEO, content strategy, and social media management to develop and execute digital marketing campaigns.
  Looking for a content writer to produce high-quality blog posts, articles, and copy for websites, with excellent research and writing skills.
  Seeking an HR manager to handle employee relations, talent acquisition, and performance management for a fast-growing tech startup.
Cluster 1:
  Hiring a financial analyst to perform data-driven analysis, generate financial reports, and assist in strategic decision-making.
  Hiring a data scientist with experience in statistical modeling, data mining, and visualization tool

In [ ]:
from sklearn.metrics import silhouette_score
valid_labels = cluster_labels != -1  # Exclude noise points
if valid_labels.sum() > 1:  # At least two samples are required to calculate the silhouette score
    silhouette_avg = silhouette_score(tfidf_vectors[valid_labels], cluster_labels[valid_labels])
    print(f"Silhouette Score: {silhouette_avg:.4f}")
else:
    print("Not enough valid labels to calculate silhouette score.")

Silhouette Score: 0.0181
